# Lab ML for Data Science: Part 2  |    **Getting Insights into Quantum-Chemical Relations**
---

> Neeraj Omprakash Chauhan, Prajna Narayan Bhat, Akash Skaria Koottungal

In [ ]:
#importing necessary packages
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import itertools
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.manifold import TSNE
import mplcursors

## 1. The QM7 Dataset

In [ ]:
file_path = './qm7.mat'
data = scipy.io.loadmat(file_path)

In [ ]:
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Mon Feb 18 17:12:08 2013',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[[36.858105 ,  2.9076326,  2.907612 , ...,  0.       ,
           0.       ,  0.       ],
         [ 2.9076326,  0.5      ,  0.29672  , ...,  0.       ,
           0.       ,  0.       ],
         [ 2.907612 ,  0.29672  ,  0.5      , ...,  0.       ,
           0.       ,  0.       ],
         ...,
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ]],
 
        [[36.858105 , 12.599944 ,  2.9019997, ...,  0.       ,
           0.       ,  0.       ],
         [12.599944 , 36.858105 ,  1.4731166, ...,  0.       ,
           0.       ,  0.       ],
         [ 2.9019997,  1.4731166,  0.5      , ...,  0.    

In [ ]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'R', 'Z', 'T', 'P'])

In [ ]:
R = data['R']
Z = data['Z']
T = data['T'].flatten()

In [ ]:
print("X:",data['X'].shape)
print("R (3d coordinates of each atom):", R.shape)
print("Z (Atomic number of each atom/molecule):", Z.shape)
print("T (Atomization enery of each molecule):", T.shape)

X: (7165, 23, 23)
R (3d coordinates of each atom): (7165, 23, 3)
Z (Atomic number of each atom/molecule): (7165, 23)
T (Atomization enery of each molecule): (7165,)


### 1.1 Visualizing Molecules

In [ ]:
R[0].shape
print("x, y, z coordinates for first molecule\n")
for x, y, z in R[0]:
  print(x,y,z)


x, y, z coordinates for first molecule

1.886438 -0.0046487264 -0.008239206
3.9499245 -0.0045920345 0.007823466
1.1976895 1.9404842 0.007823466
1.1849339 -0.99726516 1.6593875
1.2119948 -0.9589793 -1.710958
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0


In [ ]:
molecule = R[7100]
atomic_numbers = Z[7100]

threshold_distance = 3

plt.figure(figsize=(8, 8))

# Scatter plot with colors representing different atoms
colors = {1: 'blue', 6: 'black', 7: 'green', 8: 'red', 16: 'yellow'}
atom_labels = {1: 'H', 6: 'C', 7: 'N', 8: 'O', 16: 'S'}

num_atoms = len(molecule)
for i in range(min(num_atoms, len(atomic_numbers))):
    atom = molecule[i]
    if atomic_numbers[i] > 0:
        plt.scatter(atom[0], atom[1], s=100, c=colors.get(atomic_numbers[i], 'grey'), zorder=2)
        plt.text(atom[0], atom[1], atom_labels.get(atomic_numbers[i], '?'), fontsize=12, ha='right')

# Connect nearby atoms
for i in range(num_atoms):
    for j in range(i + 1, num_atoms):
        if i < len(atomic_numbers) and j < len(atomic_numbers):
            if atomic_numbers[i] > 0 and atomic_numbers[j] > 0:  # Only consider real atoms
                distance = np.linalg.norm(molecule[i] - molecule[j])
                if distance < threshold_distance:
                    plt.plot([molecule[i, 0], molecule[j, 0]],
                             [molecule[i, 1], molecule[j, 1]],
                             'k-', zorder=1)

plt.xlim(molecule[:, 0].min() - 1, molecule[:, 0].max() + 1)
plt.ylim(molecule[:, 1].min() - 1, molecule[:, 1].max() + 1)
plt.xlabel('X-coordinate')
plt.ylabel('Y-coordinate')
plt.title('2D Molecular Visualization')
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

# Colors for different atoms
colors = {1: 'blue', 6: 'black', 7: 'green', 8: 'red', 16: 'yellow'}
atom_labels = {1: 'H', 6: 'C', 7: 'N', 8: 'O', 16: 'S'}

# Scatter plot with colors representing different atoms
num_atoms = len(molecule)
for i in range(min(num_atoms, len(atomic_numbers))):
    atom = molecule[i]
    if atomic_numbers[i] > 0:  # Ignore padding zeros
        ax.scatter(atom[0], atom[1], atom[2], s=100, c=colors.get(atomic_numbers[i], 'grey'), zorder=2)
        ax.text(atom[0], atom[1], atom[2], atom_labels.get(atomic_numbers[i], '?'), fontsize=12, ha='right')

for i in range(num_atoms):
    for j in range(i + 1, num_atoms):
        if i < len(atomic_numbers) and j < len(atomic_numbers):
            if atomic_numbers[i] > 0 and atomic_numbers[j] > 0:  # Only consider real atoms
                distance = np.linalg.norm(molecule[i] - molecule[j])
                if distance < threshold_distance:
                    ax.plot([molecule[i, 0], molecule[j, 0]],
                            [molecule[i, 1], molecule[j, 1]],
                            [molecule[i, 2], molecule[j, 2]],
                            'k-', zorder=1)

ax.set_xlim(molecule[:, 0].min() - 1, molecule[:, 0].max() + 1)
ax.set_ylim(molecule[:, 1].min() - 1, molecule[:, 1].max() + 1)
ax.set_zlim(molecule[:, 2].min() - 1, molecule[:, 2].max() + 1)
ax.set_xlabel('X-coordinate')
ax.set_ylabel('Y-coordinate')
ax.set_zlabel('Z-coordinate')
ax.set_title('3D Molecular Visualization')

plt.show()

<IPython.core.display.Javascript object>

## 2. Data Representation, ML Model and Explanations

### 2.1 Data Representation

In [ ]:
atomic_numbers = []

# Define the atomic numbers present in the dataset
for molecule in Z:
    for atomic_num in molecule:
        if atomic_num not in atomic_numbers and atomic_num != 0.0:
            atomic_numbers.append(atomic_num)

# Create a mapping from atomic number to index in the one-hot encoding
# (0, 1) → Hydrogen (H)
# (1, 6) → Carbon (C)
# (2, 7) → Nitrogen (N)
# (3, 8) → Oxygen (O)
# (4, 16) → Sulfur (S)

atomic_number_to_index = {num: i for i, num in enumerate(atomic_numbers)}
index_to_atomic_number = {i: num for i, num in enumerate(atomic_numbers)}

# Function to convert atomic number to one-hot encoded vector
def one_hot_encode(atomic_num):
    vec = np.zeros(len(atomic_numbers))
    if atomic_num in atomic_number_to_index:
        vec[atomic_number_to_index[atomic_num]] = 1
    return vec


# Compute the overall vector representation for each molecule
def compute_molecular_representation(Z):
    num_molecules, max_atoms = Z.shape
    h = len(atomic_numbers)  # Length of one-hot encoded vector
    reps = []
    representations = np.zeros((num_molecules, h))
    for i in range(num_molecules):
        rep = []
        for j in range(max_atoms):
            atomic_num = Z[i, j]
            if atomic_num > 0:  # Ignore zeros which indicate no atom
                temp = one_hot_encode(atomic_num)
                representations[i] += temp
                rep.append(list(temp))
        reps.append(np.array(rep))
    return representations, reps

# Compute the molecular representations
X , contribs = compute_molecular_representation(Z)

# Print the shape of the resulting molecular representations
print(f"Shape of molecular representations: {X.shape}")
print("Vector representations\n",X[:5])

Shape of molecular representations: (7165, 5)
Vector representations
 [[1. 4. 0. 0. 0.]
 [2. 6. 0. 0. 0.]
 [2. 4. 0. 0. 0.]
 [2. 2. 0. 0. 0.]
 [2. 6. 1. 0. 0.]]


### 2.2 Ridge Regression Model

In [ ]:
def prep_data(X,T):
    X_mean = X.mean(axis=0)
    T_mean = T.mean()
    X_centered = X - X_mean
    T_centered = T - T_mean

    # Compute the covariance matrices
    Sigma_xx = np.dot(X_centered.T, X_centered) / X_centered.shape[0]
    Sigma_xt = np.dot(X_centered.T, T_centered) / X_centered.shape[0]

    return Sigma_xx,Sigma_xt,X_centered,T_centered

In [ ]:
def ridge_regression(Sigma_xx, Sigma_xt, lambda_):
    I = np.eye(Sigma_xx.shape[0])
    w = np.linalg.inv(Sigma_xx + (lambda_ * I)).dot(Sigma_xt)
    return w

# Example usage with lambda = 1.0
lambda_ = 1
Sigma_xx,Sigma_xt,X_centered,T_centered=prep_data(X,T)
w = ridge_regression(Sigma_xx, Sigma_xt, lambda_)
f_x = w.T.dot(X_centered.T)
print("Mean absolute error :", f"{(mean_absolute_error(T_centered, f_x)):.3f}")

Mean absolute error : 47.125


In [ ]:
def cross_validate_ridge(X, T, lambdas, k=10):
    kf = KFold(n_splits=k)
    errors = []
    weights = []

    for lambda_ in lambdas:
        fold_errors = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            T_train, T_test = T[train_index], T[test_index]

            # Prepare the data
            Sigma_xx_train,Sigma_xt_train,_,_ = prep_data(X_train,T_train)

            # Compute the ridge regression weights
            w = ridge_regression(Sigma_xx_train, Sigma_xt_train, lambda_)

            # Predict on the test set
            X_test_centered = X_test - X_train.mean(axis=0)
            T_test_centered_pred = np.dot(X_test_centered, w)
            T_test_pred = T_test_centered_pred + T_train.mean()

            # Calculate the error
            error = mean_absolute_error(T_test, T_test_pred)
            fold_errors.append(error)

        avg_error = np.mean(fold_errors)
        errors.append(avg_error)
        weights.append(w)

    best_lambda_index = np.argmin(errors)
    best_lambda = lambdas[best_lambda_index]
    best_error = errors[best_lambda_index]
    best_weight = weights[best_lambda_index]

    return best_lambda, best_error, errors, best_weight

In [ ]:
# Example usage
lambdas = np.logspace(-10, 10, 21)
best_lambda, best_error, errors, weights = cross_validate_ridge(X, T, lambdas)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(lambdas, errors, marker='o')
plt.xscale('log')
plt.xlabel('Lambda')
plt.ylabel('Cross-Validation Error')
plt.title('Cross-Validation Error vs. Lambda')
plt.axvline(best_lambda, color='r', linestyle='--', label=f'Best Lambda = {best_lambda:.2e}')
plt.legend()
plt.grid(True)
plt.show()

print(f'Best lambda: {best_lambda} with error: {best_error:.3f}')

<IPython.core.display.Javascript object>

Best lambda: 1e-05 with error: 15.866


###2.3 Deeper Insights with Explanations

In [ ]:
one_hots=[]
one_hots_atoms=[]
index_to_atomic_number = {i: num for i, num in enumerate(atomic_numbers)}
for i in range(len(data['R'])):
    ik=[]
    an = []
    for j in range(len(contribs[i])):
        ik.append(abs(weights.dot(contribs[i][j])))
        an.append(atom_labels[index_to_atomic_number[np.nonzero(contribs[i][j])[0][0]]])
    one_hots.append(ik)
    one_hots_atoms.append(an)

In [ ]:
%matplotlib notebook

x = data['R'][:, 0]
y = data['R'][:, 1]
z = data['R'][:, 2]

# Visualize the t-SNE representation with a bar plot on the side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9.5, 5))
scatter = ax1.scatter(x[:5], y[:5],
                      c=T[:15], cmap='cool', alpha=0.2)
ax1.set_title('t-SNE representation of Molecules')
ax2.set_title('Contributions of Atoms')

# Function to update the bar plot based on the hovered point
def update_bar_plot(cursor):
    index = cursor.index
    x = int(index)
    values = one_hots[x]

    atom_values = one_hots_atoms[x]
    ax2.clear()

    bar_plot = ax2.bar(np.arange(len(atom_values)), values)
    ax2.set_xticklabels(atom_values)
    ax2.set_xticks(np.arange(len(atom_values)))

    for i, bar in enumerate(bar_plot):
        bar.set_height(values[i])


    txt = f'data point: {x}'
    ann = cursor.annotation
    ann.set_text(txt)
    plt.draw()

# Connect the cursor event to the update_bar_plot function
mplcursors.cursor(scatter, hover=True).connect("add", update_bar_plot)

plt.colorbar(scatter, ax=ax1)
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

### 3.2 Models with Pairs of Atoms
Decomposing each molecule into its set of pairs of atoms, i.e. Ei now denotes a pair of atoms, and a molecule contains (#atoms·(#atoms−1))/2 such pairs. Generate a one-hot encoding of distances, i.e. binning them into mutiple intervals [θ1, θ2], [θ2, θ3], . . . , [θm−1, θm], and generating the vector:

\begin{equation}
{𝜙^A(𝜀_i)} =  \begin{pmatrix} I(θ1 ≤ dist(𝜀_i) < θ2) \\ I(θ2 ≤ dist(𝜀_i) < θ3) \\ \vdots \\ I(θm−1 ≤ dist(𝜀_i) < θm \end{pmatrix}
\end{equation}

Incorporating atom types, one can generate another feature representation of atom types:

\begin{equation}
{𝜙^B(𝜀_i)} =  \begin{pmatrix} I(type(𝜀_i) = HH) \\ I(type(𝜀_i) = HC) \\ I(type(𝜀_i) = HN) \\ \vdots \\ I(type(𝜀_i) = SS) \end{pmatrix}
\end{equation}

i.e. a feature map of 6 · 5/2 = 15 dimensions. Here, we have considered unordered pairs of atom types in order to implemenent invariance to the indexing.

Lastly, the feature maps φA and φB can be combined into one ‘big’ feature map consisting e.g. of all products between elements of the two feature maps, i.e. a matrix 𝜙(𝜀_i) with elements
\begin{equation}
[𝜙 (𝜺_i)]_{jk} = 𝜙^A_j (𝜺_i) · 𝜙^B_k (𝜺_i)
\end{equation}
for all j and k.

In [ ]:
molecules = R
atomic_numbers = Z

#Function to generate pairs of atoms with distances and types
def generate_pairs(molecule_coords, atomic_numbers):
    num_atoms = len(molecule_coords)
    pairs = []
    for (i, j) in itertools.combinations_with_replacement(range(num_atoms), 2):
        if atomic_numbers[i] > 0 and atomic_numbers[j] > 0:  # Only consider valid atoms
            dist = np.linalg.norm(molecule_coords[i] - molecule_coords[j])
            atom_pair = tuple(((atomic_numbers[i], atomic_numbers[j])))
            atom_type_pair = str(atom_labels[atom_pair[0]])+str(atom_labels[atom_pair[1]])
            pairs.append(((i, j), dist, atom_pair,atom_type_pair))
    return pairs

all_pairs = []
in_between_distance_for_all = []

for i in range(len(molecules)):
    in_between_distance = []
    pairs = generate_pairs(data['R'][i], data['Z'][i])
    in_between_distance = [dist[1] for dist in pairs]
    all_pairs.append(pairs)
    in_between_distance_for_all.append((in_between_distance))

In [ ]:
# Function to generate one-hot encoding for atom pairs
def generate_one_hot(atom_pair):
    one_hot = np.zeros(len(atom_pairs))
    if atom_pair in atom_pairs:
        index = atom_pairs.index(atom_pair)
        one_hot[index] = 1
    return one_hot


atom_types = list(atomic_number_to_index.keys())
atom_pairs = list(itertools.combinations_with_replacement(atom_types, 2))
# Generate one-hot encoding for all pairs
o_h_e_for_all = []

for pairs in all_pairs:
    o_h_e_molecule = []
    for pair in pairs:
        atom_pair = pair[2]
        one_hot_encoding = generate_one_hot(atom_pair)
        o_h_e_molecule.append(one_hot_encoding)

    o_h_e_for_all.append(o_h_e_molecule)

# Example: print pairs and one-hot encodings for the first molecule
print("Pairs for the first molecule:")
for pair in all_pairs[0]:
    print(pair)

print("\nOne-hot encodings for the first molecule:")
for encoding in o_h_e_for_all[0]:
    print(encoding)

Pairs for the first molecule:
((0, 0), 0.0, (6.0, 6.0), 'CC')
((0, 1), 2.063549, (6.0, 1.0), 'CH')
((0, 2), 2.0635345, (6.0, 1.0), 'CH')
((0, 3), 2.0635827, (6.0, 1.0), 'CH')
((0, 4), 2.0651567, (6.0, 1.0), 'CH')
((1, 1), 0.0, (1.0, 1.0), 'HH')
((1, 2), 3.3701808, (1.0, 1.0), 'HH')
((1, 3), 3.3701982, (1.0, 1.0), 'HH')
((1, 4), 3.3706563, (1.0, 1.0), 'HH')
((2, 2), 0.0, (1.0, 1.0), 'HH')
((2, 3), 3.3701925, (1.0, 1.0), 'HH')
((2, 4), 3.3706532, (1.0, 1.0), 'HH')
((3, 3), 0.0, (1.0, 1.0), 'HH')
((3, 4), 3.3706717, (1.0, 1.0), 'HH')
((4, 4), 0.0, (1.0, 1.0), 'HH')

One-hot encodings for the first molecule:
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0.

In [ ]:
# Function to generate one-hot encoding for distance intervals
def generate_distance_one_hot(distance, intervals):
    one_hot_distance = np.zeros(len(intervals))
    for i, (start, end) in enumerate(intervals):
        if start <= distance < end:
            one_hot_distance[i] = 1
            break
    return one_hot_distance

# Function to generate Gaussian-based distance encoding
def generate_distance_encoding(distance, intervals, sigma):
    encoding = np.zeros(len(intervals))
    for i, (start, end) in enumerate(intervals):
        center = (start + end) / 2
        encoding[i] = np.exp(-0.5 * ((distance - center) ** 2) / (sigma ** 2))
    return encoding

# Calculate min and max distances across all molecules
min_dist_between_atoms = min(min(in_between_distance_for_all))
max_dist_between_atoms = max(max(in_between_distance_for_all))

# Generate intervals for distances
points = np.linspace(0, max_dist_between_atoms-min_dist_between_atoms, 16)
intervals = [(points[i], points[i + 1]) for i in range(15)]

In [ ]:
print("Intervals :\n")
intervals

Intervals :



[(0.0, 0.5955820719401042),
 (0.5955820719401042, 1.1911641438802083),
 (1.1911641438802083, 1.7867462158203125),
 (1.7867462158203125, 2.3823282877604166),
 (2.3823282877604166, 2.9779103597005205),
 (2.9779103597005205, 3.573492431640625),
 (3.573492431640625, 4.169074503580729),
 (4.169074503580729, 4.764656575520833),
 (4.764656575520833, 5.360238647460937),
 (5.360238647460937, 5.955820719401041),
 (5.955820719401041, 6.551402791341146),
 (6.551402791341146, 7.14698486328125),
 (7.14698486328125, 7.742566935221354),
 (7.742566935221354, 8.338149007161459),
 (8.338149007161459, 8.933731079101562)]

In [ ]:
combined_features_for_all = []
distance_one_hot_for_all = []
all_atom_pair_types = []
gauss_combined_features_for_all = []
all_one_hot_encodings = []
sigma = 1.0

for pairs in all_pairs:
    combined_features_molecule = []
    distance_one_hot_for_molecule = []
    gauss_combined_features_molecule = []
    one_hot_encodes = []
    atom_pair_type = []

    for pair in pairs:
        distance = pair[1]
        atom_pair = pair[2]
        atom_pair_type.append(pair[3])

        # Generate distance one-hot encoding
        distance_one_hot = generate_distance_one_hot(distance, intervals)

        # Generate Gaussian-based distance encoding
        gauss_distance_encoding = generate_distance_encoding(distance, intervals, sigma)

        # Generate atom type one-hot encoding
        one_hot_encoding = generate_one_hot(atom_pair)
        one_hot_encodes.append(one_hot_encoding)

        # Combine both encodings
        combined_encoding = np.outer(distance_one_hot, one_hot_encoding).flatten()
        combined_features_molecule.append(np.array(combined_encoding))
        distance_one_hot_for_molecule.append(np.array(distance_one_hot))

        # Combine both encodings gaussian
        gauss_combined_encoding = np.outer(gauss_distance_encoding, one_hot_encoding).flatten()
        gauss_combined_features_molecule.append(gauss_combined_encoding)

    all_atom_pair_types.append(atom_pair_type)
    all_one_hot_encodings.append(np.array(one_hot_encodes))
    gauss_combined_features_for_all.append(np.array(gauss_combined_features_molecule))

    distance_one_hot_for_all.append(np.array(distance_one_hot_for_molecule))
    combined_features_for_all.append(np.array(combined_features_molecule))

In [ ]:
# Aggregate the combined features for each molecule
X_features = []
for molecule_features in combined_features_for_all:
    molecule_aggregated = molecule_features.sum(axis=0)  # Sum the feature vectors for all pairs in the molecule
    X_features.append(molecule_aggregated)
X_features = np.array(X_features)
X_features.shape

(7165, 225)

In [ ]:
# Example usage
lambdas = np.logspace(-10, 10, 21)
best_lambda, best_error, errors, weights_h = cross_validate_ridge(X_features, T, lambdas)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(lambdas, errors, marker='o')
plt.xscale('log')
plt.xlabel('Lambda')
plt.ylabel('Cross-Validation Error')
plt.title('Cross-Validation Error vs. Lambda')
plt.axvline(best_lambda, color='r', linestyle=':', label=f'Best Lambda = {best_lambda:.2e}')
plt.legend()
plt.grid(True)
plt.show()

print(f'Best lambda: {best_lambda} with error: {best_error}')

<IPython.core.display.Javascript object>

Best lambda: 1e-05 with error: 6.533465484733204


In [ ]:
# Aggregate the combined features for each molecule
X_features = []
for molecule_features in gauss_combined_features_for_all:
    molecule_aggregated = molecule_features.sum(axis=0)  # Sum the feature vectors for all pairs in the molecule
    X_features.append(molecule_aggregated)
X_features = np.array(X_features)


In [ ]:
# Example usage
lambdas = np.logspace(-10, 10, 21)
best_lambda, best_error, errors, weights_g = cross_validate_ridge(X_features, T, lambdas)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(lambdas, errors, marker='o')
plt.xscale('log')
plt.xlabel('Lambda')
plt.ylabel('Cross-Validation Error')
plt.title('Cross-Validation Error vs. Lambda')
plt.axvline(best_lambda, color='r', linestyle=':', label=f'Best Lambda = {best_lambda:.2e}')
plt.legend()
plt.grid(True)
plt.show()

print(f'Best lambda: {best_lambda} with error: {best_error}')

<IPython.core.display.Javascript object>

Best lambda: 1e-07 with error: 6.1924710717927605


In [ ]:
phi_explained = []
for i in range(len(combined_features_for_all)):
    phi_explained.append(np.dot(combined_features_for_all[i], weights_g))
phi_explained = np.array(phi_explained, dtype=object)

In [ ]:
# Visualize the t-SNE representation with a bar plot on the side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9.5, 5))
scatter = ax1.scatter(x[:5], y[:5],
                      c=T[:15], cmap='cool', alpha=0.2)
ax1.set_title('t-SNE representation of molecules')
ax2.set_title('Contributions of pairs of atoms')


# Function to update the bar plot based on the hovered point
def update_bar_plot(cursor):
    index = cursor.index
    x = int(index)
    values = abs(phi_explained[x])

    atom_values = all_atom_pair_types[x]
    ax2.clear()

    bar_plot = ax2.bar(np.arange(len(atom_values)), values)
    ax2.set_xticklabels(atom_values,rotation=45)
    ax2.set_xticks(np.arange(len(atom_values)))

    for i, bar in enumerate(bar_plot):
        bar.set_height(values[i])

    txt = f'data point: {x}'
    ann = cursor.annotation
    ann.set_text(txt)
    plt.draw()

# Connect the cursor event to the update_bar_plot function
mplcursors.cursor(scatter, hover=True).connect("add", update_bar_plot)

plt.colorbar(scatter, ax=ax1)
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>